In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# BigQuery DataFrames ML: 藥物名稱生成

<table align="left">
  <td>
    <a href="https://colab.sandbox.google.com/github/doggy8088/generative-ai/blob/main/language/use-cases/applying-llms-to-data/bq_dataframes_ml_drug_name_generation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab 商標"> 在 Colab 中執行
    </a>
  </td>
  <td>
    <a href="https://github.com/doggy8088/generative-ai/blob/main/language/use-cases/applying-llms-to-data/bq_dataframes_ml_drug_name_generation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub 商標"> 在 GitHub 上檢視
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/language/use-cases/applying-llms-to-data/bq_dataframes_ml_drug_name_generation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI 商標"> 在 Vertex AI Workbench 中開啟
    </a>
  </td>                    
</table>


| | |
|-|-|
|作者 | [Ashley Xu](https://github.com/ashleyxuu) |


**_注意_** : 已在下列環境中測試過這個筆記本：

* Python 版本 = 3.9


## 概觀

本筆記本的目標是要展示企業生成式 AI 使用案例。行銷使用者可以提供有關新藥物及其一般名稱的資訊，然後取得該藥物的行銷導向品牌名稱點子。

進一步了解 [BigQuery 資料架構](https://cloud.google.com/bigquery/docs/dataframes-quickstart)。


### 目標

在本教學課程中，你將瞭解生成式 AI 概念，如提示和少樣本學習，以及如何使用 BigFrames ML 僅使用直觀的資料架構 API 來執行這些任務。

執行的步驟包括：

1. 詢問使用者該藥物的通用名稱和用法。
2. 使用 `bigframes` 查詢包含 100,000 多種藥物的 FDA 資料集，並篩選品牌名稱、通用名稱和適應症與用法欄。
3. 篩選此資料集以找出可用作提示調整範例的典型品牌名稱。
4. 使用使用者輸入、一般說明、範例和反例，針對所需的品牌名稱建立提示。
5. 使用 `bigframes.ml.llm.PaLM2TextGenerator` 來產生品牌名稱選項。


### 資料集

此筆記本使用 [`bigquery-public-data.fda_drug`](https://console.cloud.google.com/bigquery?ws=!1m4!1m3!3m2!1sbigquery-public-data!2sfda_drug) 可取得的 [FDA 資料集](https://cloud.google.com/blog/topics/healthcare-life-sciences/fda-mystudies-comes-to-google-cloud)。


### 成本

本課程使用 Google Cloud 的計費元件：

* BigQuery (運算) 
* BigQuery ML

深入了解 [BigQuery 運算定價](https://cloud.google.com/bigquery/pricing#analysis_pricing_models)
和 [BigQuery ML 定價](https://cloud.google.com/bigquery/pricing#bqml)，
並使用 [定價計算工具](https://cloud.google.com/products/calculator/)
根據預計用量產生成本估計。


## 安裝

安裝執行此筆記本所需的以下套件。


In [ ]:
!pip install -U --quiet bigframes

### 僅 Colab：取消以下Cell註解，重新啟動核心。


In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 匯入函式庫


In [ ]:
import bigframes.pandas as bpd
from bigframes.ml.llm import PaLM2TextGenerator
from google.cloud import bigquery_connection_v1 as bq_connection
from IPython.display import Markdown

### 驗證你的 Google Cloud 帳戶

視你的 Jupyter 環境而定，你可能必須手動驗證。請按照以下相關說明進行操作。


**1. Vertex AI Workbench** 
* 不做任何事情，因為你已驗證身份。


**2. Local JupyterLab 執行個體 (移除註解並執行)：** 


In [ ]:
# ! gcloud auth login

**3. Colab, 取消註釋並執行：** 


In [ ]:
# from google.colab import auth

# auth.authenticate_user()

## 在開始之前

### 設定你的 Google Cloud 專案

**以下步驟為必要步驟，不論你的筆記本環境為何。** 

1. [選擇或建立 Google Cloud 專案](https://console.cloud.google.com/cloud-resource-manager)。當你首次建立帳戶時，你會獲得 300 美金的免費額度以支付你的運算/儲存費用。

2. [確定你的專案已啟用帳務](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [啟用 BigQuery API](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com)。

4. 如果你在本機執行這個筆記本，你需要安裝 [Cloud SDK](https://cloud.google.com/sdk)。


#### 設定你的專案 ID

**如果你不知道自己的專案 ID** , 請嘗試以下動作：
* 執行 `gcloud config list`。
* 執行 `gcloud projects list`。
* 參閱支援頁面：[尋找專案 ID](https://support.google.com/googleapi/answer/7014113)


In [ ]:
PROJECT_ID = "<your-project-id>"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### BigFrames 組態

接下來，我們將指定一個 [BigQuery 連線](https://cloud.google.com/bigquery/docs/working-with-connections)。如果你已經有連線，你可以簡化提供名稱並略過以下的建立步驟。


In [ ]:
# Please fill in these values.
LOCATION = "us"  # @param {type:"string"}
CONNECTION = "bigframes-ml"  # @param {type:"string"}

connection_name = f"{PROJECT_ID}.{LOCATION}.{CONNECTION}"

我們現在會嘗試使用提供的連接，如果它不存在，則建立一個新的。我們還將列印使用的服務帳戶。


In [ ]:
# Initialize client and set request parameters
client = bq_connection.ConnectionServiceClient()
new_conn_parent = f"projects/{PROJECT_ID}/locations/{LOCATION}"
exists_conn_parent = (
    f"projects/{PROJECT_ID}/locations/{LOCATION}/connections/{CONNECTION}"
)
cloud_resource_properties = bq_connection.CloudResourceProperties({})

# Try to connect using provided connection
try:
    request = client.get_connection(
        request=bq_connection.GetConnectionRequest(name=exists_conn_parent)
    )
    CONN_SERVICE_ACCOUNT = f"serviceAccount:{request.cloud_resource.service_account_id}"
# Create a new connection on error
except Exception:
    connection = bq_connection.types.Connection(
        {"friendly_name": CONNECTION, "cloud_resource": cloud_resource_properties}
    )
    request = bq_connection.CreateConnectionRequest(
        {
            "parent": new_conn_parent,
            "connection_id": CONNECTION,
            "connection": connection,
        }
    )
    response = client.create_connection(request)
    CONN_SERVICE_ACCOUNT = (
        f"serviceAccount:{response.cloud_resource.service_account_id}"
    )
# Set service account permissions
!gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member={CONN_SERVICE_ACCOUNT} --role='roles/bigquery.connectionUser'
!gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member={CONN_SERVICE_ACCOUNT} --role='roles/aiplatform.user'
!gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member={CONN_SERVICE_ACCOUNT} --role='roles/run.invoker'

print(CONN_SERVICE_ACCOUNT)

### 初始化 BigFrames 用戶端

在這裡，我們根據提供的參數設定專案組態。


In [ ]:
bpd.options.bigquery.project = PROJECT_ID
bpd.options.bigquery.location = LOCATION

## 生成一個名稱

讓我們從輸入藥物的通用名稱和說明開始。


In [ ]:
GENERIC_NAME = "Entropofloxacin"  # @param {type:"string"}
USAGE = "Entropofloxacin is a fluoroquinolone antibiotic that is used to treat a variety of bacterial infections, including: pneumonia, streptococcus infections, salmonella infections, escherichia coli infections, and pseudomonas aeruginosa infections It is taken by mouth or by injection. The dosage and frequency of administration will vary depending on the type of infection being treated. It should be taken for the full course of treatment, even if symptoms improve after a few days. Stopping the medication early may increase the risk of the infection coming back."  # @param {type:"string"}
NUM_NAMES = 10  # @param {type:"integer"}
TEMPERATURE = 0.5  # @param {type: "number"}

現在，我們可以建立提示字串，並加入名稱和描述。


In [ ]:
zero_shot_prompt = f"""Provide {NUM_NAMES} unique and modern brand names in Markdown bullet point format. Do not provide any additional explanation.

Be creative with the brand names. Don't use English words directly; use variants or invented words.

The generic name is: {GENERIC_NAME}

The indications and usage are: {USAGE}."""

print(zero_shot_prompt)

接下來，我們來建立一個輔助函式來使用我們的模型預測。它會輸入一個字串，然後將其加入暫時性 BigFrames `DataFrame`。它也會回傳從回應 `DataFrame` 中萃取出的字串。


In [ ]:
def predict(prompt: str, temperature: float = TEMPERATURE) -> str:
    # Create dataframe
    input = bpd.DataFrame(
        {
            "prompt": [prompt],
        }
    )

    # Return response
    return model.predict(input, temperature).ml_generate_text_llm_result.iloc[0]

我們現在可以初始化該模型，並獲得對提示的回應！


In [ ]:
# Get BigFrames session
session = bpd.get_global_session()

# Define the model
model = PaLM2TextGenerator(session=session, connection_name=connection_name)

# Invoke LLM with prompt
response = predict(zero_shot_prompt)

# Print results as Markdown
Markdown(response)

我們有一個好的開端了！讓我們看看我們是否可以優化我們的回應。


## 少樣本學習

讓我們嘗試使用 [少樣本學習](https://paperswithcode.com/task/few-shot-learning)。我們將提供我們要尋找內容的一些範例以及提示。

我們的提示將包含 3 個部分：
* 一般說明 (例如產生 $n$ 個品牌名稱) 
* 多個範例
* 我們要為其產生名稱的藥物資訊

讓我們了解如何建構此提示。

我們的第一個步驟是定義我們要提供多少範例到提示中。


In [ ]:
# Specify number of examples to include

NUM_EXAMPLES = 3  # @param {type:"integer"}

接下來，讓我們定義一個會設定整體背景的開頭。


In [ ]:
prefix_prompt = f"""Provide {NUM_NAMES} unique and modern brand names in Markdown bullet point format, related to the drug at the bottom of this prompt.

Be creative with the brand names. Don't use English words directly; use variants or invented words.

First, we will provide {NUM_EXAMPLES} examples to help with your thought process.

Then, we will provide the generic name and usage for the drug we'd like you to generate brand names for.
"""

print(prefix_prompt)

我們的下一步將是將範例納入提示。

我們將透過查詢 BigQuery 公共數據集來開始檢索範例的原始資料。


In [ ]:
# Query 3 columns of interest from drug label dataset
df = bpd.read_gbq(
    "bigquery-public-data.fda_drug.drug_label",
    col_order=["openfda_generic_name", "openfda_brand_name", "indications_and_usage"],
)

# Exclude any rows with missing data
df = df.dropna()

# Drop duplicate rows
df = df.drop_duplicates()

# Print values
df.head()

現在讓我們過濾結果以移除非典型名稱


In [ ]:
# Remove names with spaces
df = df[df["openfda_brand_name"].str.find(" ") == -1]

# Remove names with 5 or fewer characters
df = df[df["openfda_brand_name"].str.len() > 5]

# Remove names where the generic and brand name match (case-insensitive)
df = df[df["openfda_generic_name"].str.lower() != df["openfda_brand_name"].str.lower()]

讓我們採取 `NUM_EXAMPLES`範例包含在提示中。


In [ ]:
# Take a sample and convert to a Pandas dataframe for local usage.
df_examples = df.sample(NUM_EXAMPLES, random_state=3).to_pandas()

df_examples

讓我們現在將數據轉換為 JSON 結構，以便融入提示中。基於一致性考量，我們將對每個範例品牌名稱進行大寫。


In [ ]:
examples = [
    {
        "brand_name": brand_name.capitalize(),
        "generic_name": generic_name,
        "usage": usage,
    }
    for brand_name, generic_name, usage in zip(
        df_examples["openfda_brand_name"],
        df_examples["openfda_generic_name"],
        df_examples["indications_and_usage"],
    )
]

print(examples)

我們將為每個範例建立一個提示範本，並檢視第一個範本。


In [ ]:
example_prompt = ""
for example in examples:
    example_prompt += f"Generic name: {example['generic_name']}\nUsage: {example['usage']}\nBrand name: {example['brand_name']}\n\n"

example_prompt

最後，我們可以在提示訊息中加入一個字尾。它會包含藥物的通用名稱、使用方式，最後要求提供品牌名稱。


In [ ]:
suffix_prompt = f"""Generic name: {GENERIC_NAME}
Usage: {USAGE}
Brand names:"""

print(suffix_prompt)

讓我們把它完全融合成幾個提示短語。


In [ ]:
# Define the prompt
few_shot_prompt = prefix_prompt + example_prompt + suffix_prompt

# Print the prompt
print(few_shot_prompt)

現在讓我們將提示傳遞給 LLM，以得到回應！


In [ ]:
response = predict(few_shot_prompt)

Markdown(response)

# 大量產生

讓我們將這些實驗提升至下一個層級，大量產生許多名稱。我們將看到如何大規模利用 BigFrames！

我們可以從找出沒有品牌名稱的藥物開始。大約有 4,000 種藥物符合此標準。我們將在此筆記本中設定 100 的限制。


In [ ]:
# Query 3 columns of interest from drug label dataset
df_missing = bpd.read_gbq(
    "bigquery-public-data.fda_drug.drug_label",
    col_order=["openfda_generic_name", "openfda_brand_name", "indications_and_usage"],
)

# Exclude any rows with missing data
df_missing = df_missing.dropna()

# Include rows in which openfda_brand_name equals openfda_generic_name
df_missing = df_missing[
    df_missing["openfda_generic_name"] == df_missing["openfda_brand_name"]
]

# Limit the number of rows for demonstration purposes
df_missing = df_missing.head(100)

# Print values
df_missing.head()

我們將建立一欄 `prompt` 自訂提問給每一列。


In [ ]:
df_missing["prompt"] = (
    "Provide a unique and modern brand name related to this pharmaceutical drug."
    + "Don't use English words directly; use variants or invented words. The generic name is: "
    + df_missing["openfda_generic_name"]
    + ". The indications and usage are: "
    + df_missing["indications_and_usage"]
    + "."
)

我們將建立一個新的輔助方法，`batch_predict()`，並查詢 LLM。此工作可能需要幾分鐘才能執行。


In [ ]:
def batch_predict(
    input: bpd.DataFrame, temperature: float = TEMPERATURE
) -> bpd.DataFrame:
    return model.predict(input, temperature).ml_generate_text_llm_result


response = batch_predict(df_missing["prompt"])

讓我們來看看其中一個回應的結果！


In [ ]:
# Pick a sample
k = 0

# Gather the prompt and response details
df_missing = df_missing.head(100).reset_index(drop=True)

prompt_generic = df_missing["openfda_generic_name"][k]
prompt_usage = df_missing["indications_and_usage"][k]
response_str = response[k]

# Print details
print(f"Generic name: {prompt_generic}")
print(f"Brand name: {prompt_usage}")
print(f"Response: {response_str}")

恭喜！已學會如何使用生成式AI來跳過創意流程。

你也看到BigFrames如何可以管理流程的每一個步驟，包括收集數據、資料處理和查詢LLM。


## 清理

如需清理此專案中使用的所有 Google Cloud 資源，你可以 [刪除專案](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) 用於教學課程。

否則，你可以取消註解剩餘的Cell，然後執行它們，以刪除你在此教學課程中建立的個別資源：


In [ ]:
# Delete the BigQuery Connection
from google.cloud import bigquery_connection_v1 as bq_connection

client = bq_connection.ConnectionServiceClient()
CONNECTION_ID = f"projects/{PROJECT_ID}/locations/{LOCATION}/connections/{CONNECTION}"
client.delete_connection(name=CONNECTION_ID)
print(f"Deleted connection {CONNECTION_ID}.")